In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
# If needed, add directory names
os.chdir('/content/drive/My Drive/Code/EXTRA DATASET/movie-corpus')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import pickle
import os
import re
import json

In [ ]:
# Load in movie ids (keys) with urls (in here are the movie names)
corpus = json.load(open('corpus.json'))['url']

In [ ]:
# Load in speaker names
speakers = json.load(open('speakers.json'))
speakernames = set([speakers[key]['meta']['character_name'] for key in speakers.keys()])

In [ ]:
%%time
# The utterances file containts all needed data and can be read in easily in a pandas dataframe
df = pd.read_json('utterances.json', lines = True)
df

CPU times: user 25.8 s, sys: 2.21 s, total: 28 s
Wall time: 31.5 s


,id,conversation_id,text,speaker,meta,reply-to,timestamp,vectors
0,L1045,L1044,They do not!,u0,"{'movie_id': 'm0', 'parsed': [{'rt': 1, 'toks'...",L1044,NaT,[]
1,L1044,L1044,They do to!,u2,"{'movie_id': 'm0', 'parsed': [{'rt': 1, 'toks'...",None,NaT,[]
2,L985,L984,I hope so.,u0,"{'movie_id': 'm0', 'parsed': [{'rt': 1, 'toks'...",L984,NaT,[]
3,L984,L984,She okay?,u2,"{'movie_id': 'm0', 'parsed': [{'rt': 1, 'toks'...",None,NaT,[]
4,L925,L924,Let's go.,u0,"{'movie_id': 'm0', 'parsed': [{'rt': 0, 'toks'...",L924,NaT,[]
...,...,...,...,...,...,...,...,...
304708,L666371,L666369,Lord Chelmsford seems to want me to stay back ...,u9030,"{'movie_id': 'm616', 'parsed': [{'rt': 2, 'tok...",L666370,NaT,[]
304709,L666370,L666369,I'm to take the Sikali with the main column to...,u9034,"{'movie_id': 'm616', 'parsed': [{'rt': 1, 'tok...",L666369,NaT,[]
304710,L666369,L666369,"Your orders, Mr Vereker?",u9030,"{'movie_id': 'm616', 'parsed': [{'rt': 1, 'tok...",None,NaT,[]
304711,L666257,L666256,"Good ones, yes, Mr Vereker. Gentlemen who can ...",u9030,"{'movie_id': 'm616', 'parsed': [{'rt': 1, 'tok...",L666256,NaT,[]


In [ ]:
%%time
# Split the meta data column into a movie id with aditional column with the rest of the dict; only movie ids are needed
ser = df['meta'].apply(pd.Series)

CPU times: user 1min 11s, sys: 617 ms, total: 1min 12s
Wall time: 1min 13s


In [ ]:
df['movie_id'] = ser['movie_id']
# These columns are not needed at the moment
df = df.drop(['meta','reply-to','timestamp','vectors'],axis=1)
df

,id,conversation_id,text,speaker,movie_id
0,L1045,L1044,They do not!,u0,m0
1,L1044,L1044,They do to!,u2,m0
2,L985,L984,I hope so.,u0,m0
3,L984,L984,She okay?,u2,m0
4,L925,L924,Let's go.,u0,m0
...,...,...,...,...,...
304708,L666371,L666369,Lord Chelmsford seems to want me to stay back ...,u9030,m616
304709,L666370,L666369,I'm to take the Sikali with the main column to...,u9034,m616
304710,L666369,L666369,"Your orders, Mr Vereker?",u9030,m616
304711,L666257,L666256,"Good ones, yes, Mr Vereker. Gentlemen who can ...",u9030,m616


In [ ]:
# List of all unique movie ids, every movie id: is connected to at precisely one movie
movie_ids = df['movie_id'].unique()
len(movie_ids)

617

In [ ]:
# Create a dictionairy with movie ids as keys
# based on data analysys some ids are excluded, to see uncomment the second movies dictionary row
movies = {id : df[df['movie_id']==id] for id in movie_ids if id != 'm68' and id != 'm432' and id != 'm382' and id != 'm215' and id != 'm289' and id != 'm521' and id != 'm445'}
#movies = {id : df[df['movie_id']==id] for id in ids}
#movies

In [ ]:
len(movies)

610

In [ ]:
cd ..

/content/drive/MyDrive/Code/EXTRA DATASET


In [ ]:
keys = list(movies.keys())
frames = [movies[key] for key in keys]

In [ ]:
lens = [len(frame) for frame in frames]
sum(lens)

301045

In [ ]:
## Function to temporary remove normal/expected characters
def remnormals(sen):
    l = sen
    l = l.replace('.','').replace('?','').replace('!','').replace(',','').replace('--','').replace('-','').replace('\n','').replace(';','')
    l = l.replace(':','').replace('\"','').replace('/','').replace('`','').replace('…','').replace("'",'').replace("´",'').replace("·",'')
    l = l.replace("<",'').replace(">",'').replace("«",'').replace("»",'').replace("’",'').replace("$",'').replace("+",'')#.replace(')','')
    l = l.replace("“",'').replace("”",'').replace("£",'').replace("%",'').replace('‘','')#.replace('(','').replace("[",'').replace("]",'')

    return l

In [ ]:
%%time
# Check weird characters or letters
# Tee the outcomes, uncomment the print statements
n=-1
for file in frames:
  for sen in file['text']:
    n+=1

    if len(sen) < 3:
      #print(file['movie_id'].unique(),sen)
      stoploop = 're'

    check = sen
    l = sen
    l = remnormals(sen)

    # This way checked what strange letters and were.
    e = set(re.findall('[ÃÂâãËë]',l))

    # Larger than 1 because of ' '
    if len(e) > 1:
      #print(file['movie_id'].unique(),' sentence: ',sen,'\n These are the characters: ',e,'\n')
      stoploop = 're'

    # Other strange characters.
    e = set(re.findall('[^\w\*]',l))

    if len(e) > 1:
      #print(file['movie_id'].unique(),' sentence: ',sen,'\n These are the characters: ',e,'\n')
      stoploop = 're'

CPU times: user 1.67 s, sys: 6.82 ms, total: 1.68 s
Wall time: 1.69 s


In [ ]:
n=-1
# Check brackets because of weird occurances check ~
# Some files have brackets around the sentences, some for translations and some randomly
for frame in frames:
  frame['text'] = frame.text.str.replace('</u>','').str.replace('<u>','').str.replace('</i>','').str.replace('<i>','').str.replace('<b>','').str.replace('</b>','')
  for sen in frame['text']:
    if re.search('~',sen) != None:
      #print(frame['movie_id'].unique(),sen)
      stoploop ='re'
    if re.search('\]',sen) != None or re.search('\[',sen) != None:
#      print('\n',frame['movie_id'].unique(),sen)
      stoploop ='re'

<ipython-input-19-e359ba9952c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['text'] = frame.text.str.replace('</u>','').str.replace('<u>','').str.replace('</i>','').str.replace('<i>','').str.replace('<b>','').str.replace('</b>','')


In [ ]:
# Check if there are action instances usually indicated by capital letters speaker does x, f.e. FLORA does x
for speaker in list(speakernames):
  for frame in frames:
    for sen in frame['text']:
        if speaker !='' and speaker != 'J' and speaker != 'DA' and speaker != 'M' and speaker != 'Q' and speaker != 'STU' and speaker != 'VOICE' and speaker != 'X' and speaker != 'BERNIE' and speaker != 'ACE' and speaker in sen:
            #print(speaker,' sen',sen)
            stoploop ='re'
        if speaker == 'DEL' or speaker == 'CON' or speaker =='CO':
            #print(sen)
            stoploop ='re'
        # These show the following files need to be checked: m283, m384, m567 and m521
        if 'HERBERT' in sen or 'HONORA' in sen or 'SAM lets' in sen or 'CUT to' in sen or 'ADA pulls' in sen or 'MELTON takes' in sen or 'Claremont sits' in sen or 'WESSEX gives' in sen or 'FLORA' in sen:
            #print(sen)
            df = frame[frame['text'] == sen]
            #display(df)
        if 'LINE OF DIALOGUE CUT OFF' in sen or 'SCENE' in sen or 'DELETE' in sen or 'CONTINUED' in sen:
            #print(sen)
            stoploop ='re'

In [ ]:
# m283 en m384 m567 m521
# Checked these files on actions following data anlysis, the following one is removed because of the shortness of the file and the strange sentences
# Note to see this for this file, uncomment the second code line in the creation of the movies dict
"""
print(len(movies['m521']))
for line in movies['m521']['text']:
  print(line)
"""

"\nprint(len(movies['m521']))\nfor line in movies['m521']['text']:\n  print(line)\n"

In [ ]:
nums = [0,1,2,3,4,5,6,7,8,9]
# Find the brackets that are random / where there are action statements or noises included
# Remove either the bracket or the statement / noise
def checkbrackets(l):
  search1 = re.search('[({]',l)
  search = re.search('[)}]',l)
  hsearch = re.search('159',l)

  newsen = l
  nl = l
  # If there is one closing bracket
  if (search1 == None and search != None) or (search != None and search.span()[0]<search1.span()[0]):
    index = search.span()[0]

    # If there is a number in front of it, it is a summation
    if l[index-1] not in nums:
      nl = nl[:index]
      # Usually it is two punctuation marks before the bracket where it can be cut off
      for i in range(len(re.findall('[.?,]',nl))-1):
        insearch = re.search('[.?,]',nl).span()[1]
        nl = nl[insearch:]

      ind = re.search(nl[:7],l)
      newsen = l[:ind.span()[0]]

      # If there is text behind the bracket, include this
      if len(l[index+1:]) > 1:
        newsen = newsen + l[index+1:]

    else:
      newsen = nl

  elif search1 != None:
    index1 = search1.span()[0]
    nl = l[index1:]
    search2 = re.search('[)]',nl)
    newsen=nl

    if search2 != None:
      index2 = search2.span()[1]
      newsen=nl
      if index1 != 0:
        newsen = l[:index1-1]+l[index2+index1:]

    else:
      newsen = l[:index1]

  elif hsearch != None:
    newsen = l[:hsearch.span()[0]]


  else:
    newsen = nl

  return newsen

In [ ]:
def remsqbrackets(sen):
    newsen = sen
    search = re.search('\[',sen)
    search2 = re.search('\]',sen)
    if search != None:
      newsen = sen[:search.span()[0]]+sen[search2.span()[1]:]
    elif search == None and search2 != None:
      newsen = sen.replace(']','')

    return newsen

In [ ]:
def removebs(sen):
  search = re.search('\<b\>',sen)
  if search != None:
    return sen[:search.span()[0]]
  else:
    return sen

In [ ]:
# The <b>s here are are script mentions of places with sometimes after it peoples actions (just for this script)
df = movies['m325']
df['text'] = df['text'].apply(lambda x : removebs(x))
movies['m325'] = df

<ipython-input-47-9c3409275a45>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x : removebs(x))


In [ ]:
%%time
movies2 = {}

# These mention deleted or continued scenes or other script notes and can be removed
notinsen = ['SCENE','DELETE','CONTINUED','LINE OF DIALOGUE CUT OFF']

# Action comments, from the cell of analysis of frame/file m283 en m384 m567 m521
actions = ['SAM lets go of the parcel and pushes JILL away. She moves off.','ADA nods', 'ADA pulls up a chair','UNSUBTITLED','HERBERT shakes his head.',
           'wanders out','pulls his coat','is chopping ','waves a pair','frowns at Henry',
           'sits down next to Honora','pops a couple','whips off her','and Pauline hurriedly work','sighs as HONORA puts','He sighs',
           'looks down at the ground','stands and walks towards','steps back.','whirls around and','holds up a loaded','starts to cut',
           'comes in with a breakfast','leads a young man','glows with pride','Fruit.','turns around and Juliet','is bending down','stands and pikcs',
           'looks startled','gives her a sympathethic','hurries over','and HERBERT exchange']

# Check brackets because of weird occurances
for key in keys:
    frame = movies[key]

    frame['text'] = frame.text.str.replace('</u>','').str.replace('<u>','').str.replace('</i>','').str.replace('<i>','').str.replace('<b>',
                        '').str.replace('</b>','').str.replace('&quot;','').str.replace('\t',' ').str.replace('|','').str.replace('*'
                            ,'').str.replace('&lt;&lt;','').str.replace('&gt;&gt;','')

    frame['text'] = frame['text'].apply(lambda x : checkbrackets(x))


    if key != 'm201' and key != 'm129' and key != 'm483':
      frame['text'] = frame['text'].apply(lambda x : remsqbrackets(x))

    else:
      frame['text'] = frame.text.str.replace('[','').str.replace(']','')


    if 'T-w~e-l-v~e.' in frame['text'].values:
        frame['text'] = [sen if sen != 'T-w~e-l-v~e.' else 'Twelve.' for sen in frame['text']]

    # else do not include sen
    for sen in frame['text']:
        for n in notinsen:
            if n in sen:
                df = frame[frame['text'] == sen]
                frame = frame.drop(df.index)

        if sen == '' or sen == '.':
            df = frame[frame['text'] == sen]
            frame = frame.drop(df.index)



        # Remove action parts or action sentences
        for action in actions:
            if action in sen:
                newsen = sen[:re.search(action,sen).span()[0]]
                df = frame[frame['text'] == sen]
                if len(newsen) == 0:
                    frame = frame.drop(df.index)
                else:
                    frame.at[df.index[0], 'text'] = newsen


    # These are place statements
    if 'INT.' in sen or 'EXT.' in sen:
        df = frame[frame['text'] == sen]
        #display(df)
        frame = frame.drop(df.index)


    if len(frame) > 10:
      movies2[key] = frame

<timed exec>:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
<timed exec>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

CPU times: user 5.89 s, sys: 107 ms, total: 5.99 s
Wall time: 6.01 s


In [ ]:
movies = movies2
keys = list(movies.keys())
frames = [movies[key] for key in keys]
frames[0]

,id,conversation_id,text,speaker,movie_id
0,L1045,L1044,They do not!,u0,m0
1,L1044,L1044,They do to!,u2,m0
2,L985,L984,I hope so.,u0,m0
3,L984,L984,She okay?,u2,m0
4,L925,L924,Let's go.,u0,m0
...,...,...,...,...,...
664,L191,L191,Jesus! Can a man even grab a sandwich before ...,u11,m0
665,L171,L170,Sarah Lawrence is on the other side of the cou...,u11,m0
666,L170,L170,What's a synonym for throbbing?,u10,m0
667,L162,L161,Make anyone cry today?,u11,m0


In [ ]:
%%time
# Check weird characters or letters a test if it looks clean now
n=-1
for file in frames:
  for sen in file['text']:
    n+=1

    check = sen
    l = sen
    l = remnormals(sen)

    # This way checked what strange letters and were.
    e = set(re.findall('[ÃÂâãËë]',l))

    # Larger than 1 because of ' '
    if len(e) > 1:
      print(file['movie_id'].unique(),' sentence: ',sen,'\n These are the characters: ',e,'\n')

    # Other strange characters.
    e = set(re.findall('[^\w\*]',l))

    if len(e) > 1:
      print(file['movie_id'].unique(),' sentence: ',sen,'\n These are the characters: ',e,'\n')


['m20']  sentence:  J&B. Straight. 
 These are the characters:  {'&', ' '} 

['m20']  sentence:  Did you know that guy who disappeared? Didn't he work at Pierce & Pierce, too? Was he a friend of yours? 
 These are the characters:  {'&', ' '} 

['m21']  sentence:  'Knows I'm altering the coordinates. Let's jump to #12. 
 These are the characters:  {'#', ' '} 

['m21']  sentence:  Okay, #2. 
 These are the characters:  {'#', ' '} 

['m21']  sentence:  Delbert seems to enter #21 twice. Without leaving the first time. 
 These are the characters:  {'#', ' '} 

['m21']  sentence:  He's been altering the coordinates since we logged on. He's a step ahead. Let's jump to #12. 
 These are the characters:  {'#', ' '} 

['m21']  sentence:  Let's try #2. 
 These are the characters:  {'#', ' '} 

['m21']  sentence:  Milo? Shrot suspects somebody broke into #21. I was in his office when he was reviewing the card readouts. 
 These are the characters:  {'#', ' '} 

['m21']  sentence:  I snuck into #21. 

In [ ]:
# Create list of titles and connection to movie ids
corpuskeys = list(corpus.keys())

corpusvalsdict = {key : corpus[key].split('/')[4].split('.')[0] for key in corpuskeys}
corpusvals = list(corpusvalsdict.values())
corpusvals[:3]

# Some names had differences in there where the general rule did not apply
corpusvalsdict = {}
for key in corpuskeys:
  val = corpus[key].split('/')[4].split('.')[0]
  if val == 'ridleyscott':
    val = val+'_'+corpus[key].split('/')[6].split('.')[0]
  elif val == 'clarke':
    val = val+'_'+corpus[key].split('/')[7].split('.')[0]
  elif val == 'scripts' or val == 'Trek':
    val = corpus[key].split('/')[5].split('.')[0]

  corpusvalsdict[key] = val

corpusvals = list(corpusvalsdict.values())
len(corpusvalsdict),corpusvals[:3]

(617, ['10Things', '1492-ConquestOfParadise', '15minutes'])

In [ ]:
movies_titles = {corpusvalsdict[key]:movies[key] for key in keys}
movies_titles_keys = list(movies_titles.keys())
print(movies_titles_keys[0])
len(movies_titles),movies_titles[movies_titles_keys[0]]

10Things


(609,
         id conversation_id                                               text  \
 0    L1045           L1044                                       They do not!   
 1    L1044           L1044                                        They do to!   
 2     L985            L984                                         I hope so.   
 3     L984            L984                                          She okay?   
 4     L925            L924                                          Let's go.   
 ..     ...             ...                                                ...   
 664   L191            L191  Jesus!  Can a man even grab a sandwich before ...   
 665   L171            L170  Sarah Lawrence is on the other side of the cou...   
 666   L170            L170                    What's a synonym for throbbing?   
 667   L162            L161                             Make anyone cry today?   
 668   L161            L161                                  In the microwave.   
 
     spe

In [ ]:
%%time
corpusdict_titles = {}
# Create the next speakers column
for key in movies_titles_keys:
  df = movies_titles[key]
  next_speakers = [line for line in df.speaker]
  df['Next Speaker'] = [next_speakers[0]] + next_speakers[:-1]
  corpusdict_titles[key] = df


corpusdict_titles[key]

<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

CPU times: user 386 ms, sys: 30 ms, total: 416 ms
Wall time: 404 ms


<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,id,conversation_id,text,speaker,movie_id,Next Speaker
304640,L666463,L666462,Zulu may not wear shoes or trousers and the li...,u9025,m616,u9025
304641,L666462,L666462,Why don 't the Zulus attack?,u9026,m616,u9025
304642,L666368,L666365,One Zulu is only one man.. ..and I'm afeared o...,u9025,m616,u9026
304643,L666367,L666365,"You afeared of the Zulus then, Quartermaster?",u9026,m616,u9025
304644,L666366,L666365,Could be. Across the river into Zululand. . T...,u9025,m616,u9026
...,...,...,...,...,...,...
304708,L666371,L666369,Lord Chelmsford seems to want me to stay back ...,u9030,m616,u9034
304709,L666370,L666369,I'm to take the Sikali with the main column to...,u9034,m616,u9030
304710,L666369,L666369,"Your orders, Mr Vereker?",u9030,m616,u9034
304711,L666257,L666256,"Good ones, yes, Mr Vereker. Gentlemen who can ...",u9030,m616,u9030


## Spark frames

In [ ]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=0c98a1c69ace5b2d67b3cc763fb920ea2d48f48c07285dad3120323fa3e13bfc
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder.appName("Sparkify").config('spark.sql.session.timeZone', 'GMT+1') \
        .getOrCreate()

In [ ]:
%%time
keys = list(movies.keys())
# To Pandas is soly done for visualisation purposes
for k in keys:
  movies[k] = spark.createDataFrame(movies[k]).withColumn("Y", when(col("Speaker") == col("Next Speaker"),0).otherwise(1))\
                .withColumn("Sentence Length",F.split('text',' ')).withColumn('Sentence Length',size('Sentence Length')).toPandas()

movies[keys[0]]

CPU times: user 29.9 s, sys: 1.85 s, total: 31.8 s
Wall time: 4min 53s


,id,conversation_id,text,speaker,movie_id,Next Speaker,Y,Sentence Length
0,L1045,L1044,They do not!,u0,m0,u0,0,3
1,L1044,L1044,They do to!,u2,m0,u0,1,3
2,L985,L984,I hope so.,u0,m0,u2,1,3
3,L984,L984,She okay?,u2,m0,u0,1,2
4,L925,L924,Let's go.,u0,m0,u2,1,2
...,...,...,...,...,...,...,...,...
663,L191,L191,Jesus! Can a man even grab a sandwich before ...,u11,m0,u10,1,14
664,L171,L170,Sarah Lawrence is on the other side of the cou...,u11,m0,u11,0,10
665,L170,L170,What's a synonym for throbbing?,u10,m0,u11,1,5
666,L162,L161,Make anyone cry today?,u11,m0,u10,1,4


In [ ]:
# Store for later use/access
#with open('processedDS2sparks.pkl', 'wb') as fp:
#    pickle.dump(movies, fp)
#    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [ ]:
frames = [movies[key] for key in keys]
frames[0]

,id,conversation_id,text,speaker,movie_id,Next Speaker,Y,Sentence Length
0,L1045,L1044,They do not!,u0,m0,u0,0,3
1,L1044,L1044,They do to!,u2,m0,u0,1,3
2,L985,L984,I hope so.,u0,m0,u2,1,3
3,L984,L984,She okay?,u2,m0,u0,1,2
4,L925,L924,Let's go.,u0,m0,u2,1,2
...,...,...,...,...,...,...,...,...
663,L191,L191,Jesus! Can a man even grab a sandwich before ...,u11,m0,u10,1,14
664,L171,L170,Sarah Lawrence is on the other side of the cou...,u11,m0,u11,0,10
665,L170,L170,What's a synonym for throbbing?,u10,m0,u11,1,5
666,L162,L161,Make anyone cry today?,u11,m0,u10,1,4


In [ ]:
# Create one concattenated frame of alle separate frames (subtitle files of series and movies)
%%time

concat_frame = pd.DataFrame()
for frame in frames:
  concat_frame = pd.concat([concat_frame,frame],ignore_index=True)

concat_frame

CPU times: user 17.8 s, sys: 356 ms, total: 18.2 s
Wall time: 18.4 s


,id,conversation_id,text,speaker,movie_id,Next Speaker,Y,Sentence Length
0,L1045,L1044,They do not!,u0,m0,u0,0,3
1,L1044,L1044,They do to!,u2,m0,u0,1,3
2,L985,L984,I hope so.,u0,m0,u2,1,3
3,L984,L984,She okay?,u2,m0,u0,1,2
4,L925,L924,Let's go.,u0,m0,u2,1,2
...,...,...,...,...,...,...,...,...
300688,L666371,L666369,Lord Chelmsford seems to want me to stay back ...,u9030,m616,u9034,1,12
300689,L666370,L666369,I'm to take the Sikali with the main column to...,u9034,m616,u9030,1,12
300690,L666369,L666369,"Your orders, Mr Vereker?",u9030,m616,u9034,1,4
300691,L666257,L666256,"Good ones, yes, Mr Vereker. Gentlemen who can ...",u9030,m616,u9030,0,11


In [ ]:
#with open('processedDS2concattedframe.pkl', 'wb') as fp:
#    pickle.dump(concat_frame, fp)
#    print('dictionary saved successfully to file')

dictionary saved successfully to file
